In [ ]:
https://forums.burst-team.us/topic/33/burst-address-generator-v0-2-3-all-platforms
https://github.com/de-luxe/burstcoin-address-generator/releases
get 1 burst coin to create the address in the blockchain
https://faucet.burst-team.us
get the id of BURST-TALE-WK6Y-NMTE-4DB9Z from:
http://burstcoin.biz/address/3430946750618968652 paste the address in the search bar

ew miner:
git clone https://github.com/Mirkic7/mdcct
https://forums.burst-team.us/topic/1142/linux-really-need-help/3
./plotavx2 -k 3430946750618968652 -x 0 -d /tmp/burst/ -s 1 -n 7812000 -m 500 -t 1 -a

wallet:
https://github.com/burst-team/burstcoin/releases

In [ ]:
%reload_ext autoreload
%autoreload 2
from bitmex import *

#$x('//table[@id="marketBTC"]//td[2]/text()')
btc = ["XRP", "STR", "ETH", "LTC", "ETC", "XMR", "DGB", "DASH", "FCT", "DOGE", "BTS", "XEM", "GNO", "SC", "GNT", "ZEC", "STEEM", "MAID", "PASC", "SYS", "LSK", "CLAM", "STRAT", "DCR", "XCP", "REP", "NXT", "POT", "FLDC", "VTC", "NAV", "PINK", "ARDR", "GAME", "BCN", "BURST", "VRC", "BELA", "AMP", "SJCX", "LBC", "XBC", "PPC", "XVC", "GRC", "NAUT", "BTM", "OMNI", "BCY", "EXP", "EMC2", "SBD", "NOTE", "HUC", "VIA", "BLK", "NMC", "XPM", "RIC", "NXC", "RADS", "NEOS", "FLO", "BTCD"]
#$x('//table[@id="marketETH"]//td[2]/text()')
eth = ["GNO", "ETC", "GNT", "ZEC", "REP", "STEEM", "LSK"]
#$x('//table[@id="marketXMR"]//td[2]/text()')
xmr = ["LTC", "ZEC", "DASH", "NXT", "MAID", "BCN", "BTCD", "BLK"]
#$x('//table[@id="marketUSDT"]//td[2]/text()')
usdt = ["BTC", "XRP", "STR", "LTC", "ETH", "ETC", "XMR", "DASH", "ZEC", "NXT", "REP"]

#df = getPoloniexHistorical('BTC_XMR')

In [ ]:
lss = []
for i in range(len(btc)):
    lss.append('BTC_%s' % btc[i])
#lss = ['BTC_ETC', 'BTC_ETH']
print lss

In [ ]:
lss = []
btc = 'ETH BURST XMR SC'.split(' ')
for i in range(len(btc)):
    lss.append('BTC_%s' % btc[i])
print lss

In [ ]:
import numpy as n
bars=1000
lsa = 'BTC_ETH BTC_BURST BTC_SC'.split()
rdf = n.zeros([len(lsa),bars,7])
print rdf.shape

fields = 'open high low close quotedVolume volume weightedVolume'.split()
for i in range(len(lsa)):
    df1 = getPoloniexHistorical(symbol=lsa[i], period=14400, bars=bars)
    print df1.shape
    
    rdf[i,:,:] = df1.set_index('date').ix[:,fields].get_values()
#df1.dtypes
#rdf
rf = []
cc = currencyCube(len(lsa),bars,7, index=df1['date'], columns=fields, rdf=rdf)
cc

In [ ]:
#rdf = currencyCube(550,5,9, index=n.random.randn(550), columns='a b c d e f g h i'.split(' '))
#p.DataFrame(rdf['data'][0], index=rdf['index'], columns=rdf['columns'])
cc = currencyCube(r=len(lsa),tf=bars,c=7, index=df1['date'], columns=fields, rdf=rdf)
cc

In [ ]:
cc

In [ ]:
from qoreliquid import normalizeme
from qoreliquid import sigmoidme
import matplotlib.pylab as plt
mdf = p.DataFrame()
for i in lss: #[0:5]:
    print i
    df = getPoloniexHistorical(symbol=i, period=14400, bars=300)
    sdf = df.set_index('date').ix[:, 'close'.split(' ')]
    sdf[i] = sdf['close']
    sdf = sdf[[i]]
    sdf = sdf.ffill().bfill()
    mdf = mdf.combine_first(sdf)
mdf = normalizeme(mdf)
#mdf.ffill().bfill()
print mdf
#sdf.plot()
plt.plot(mdf)
plt.legend(lss)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
from pylab import rcParams
import seaborn as sns
sns.set()
%pylab inline
rcParams['figure.figsize'] = 30, 5

df = getPoloniexHistorical(symbol='BTC_ETC', period=86400, bars=300)
df.ix[:, 'open high low close'.split(' ')].plot()
#df = getPoloniexHistorical(symbol='BTC_ETC', period=240)
#df.ix[:, 'open high low close'.split(' ')].plot()
plt.show()

In [ ]:
periods = [300, 900, 1800, 7200, 14400, 86400]

In [ ]:
btc = 'DASH ETH FCT GNO LTC XMR REP XRP ZEC'.split(' ')
periods = '1 5 15 30 60 240 14400'.split(' ')
periods = [300, 900, 1800, 7200, 14400, 86400]
#periods = [1, 5, 15, 30, 60, 3600, 14400, 86400]
print periods
from matplotlib import pyplot as plt
from pylab import rcParams
import seaborn as sns
sns.set()
%pylab inline
rcParams['figure.figsize'] = 30, 5
for i in btc:
    print i
    df = getPoloniexHistorical(symbol='BTC_%s' % i, period=1)
    df.ix[:, 'open high low close'.split(' ')].plot()
    plt.show()
    print '====='

In [ ]:
# https://www.google.com.ar/search?q=coinmarketcap+ohlc&oq=coinmarketcap+ohlc&aqs=chrome..69i57.3815j0j7&client=ubuntu&sourceid=chrome&ie=UTF-8
#https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400
# https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400
    
# poloniex
import drest
api = drest.API('https://poloniex.com/')
response = api.make_request('GET', '/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400')
print response.data

#df = p.DataFrame(response.data)#.transpose()
#pf(df)
#df

In [ ]:
# bitmex
import drest
#api = drest.API('http://socket.coincap.io/')
api = drest.API('https://www.bitmex.com/api/v1')
#response = api.make_request('GET', '/trade?count=100&reverse=false')
#response = api.make_request('GET', '/instrument')
response = api.make_request('GET', '/instrument/indices')
#print response.data

df = p.DataFrame(response.data)#.transpose()
#pf(df)
df

In [ ]:
exchangePriority = {
'Bittrex':3,
'Cryptopia':1,
'Novaexchange':2,
'Poloniex':5,
'YoBit':4
}

In [ ]:
import pandas as p

# coinmarketcap
import drest
api = drest.API('http://api.coinmarketcap.com/')
response = api.make_request('GET', '/v1/ticker/')
res = response.data

c = '24h_volume_usd available_supply id last_updated market_cap_usd name percent_change_1h percent_change_24h percent_change_7d price_btc price_usd rank symbol total_supply'.split(' ')    

df = p.DataFrame(res)
df = df.set_index('symbol')

# convert to numeric
for i in c:
    try:    df[i] = p.to_numeric(df[i])
    except: ''

# filter
# https://www.youtube.com/watch?v=JF3eXDbzmg0
df = df[df['price_usd'] <= 0.1]
df = df[df['24h_volume_usd'] >= 100000]
df = df.drop('FEDS')

#print c
#print df.dtypes
c = '24h_volume_usd available_supply id market_cap_usd name percent_change_24h percent_change_7d price_btc price_usd rank symbol total_supply'.split(' ')
df = df.ix[:, c]
df = df.sort_values(by='24h_volume_usd', ascending=False)
df = df.sort_values(by='percent_change_24h', ascending=False)

# portfolio
bal = 165.11
#bal = 1000
df['portPcnt'] = df['price_usd'] / df['price_usd'].sum() * 1
#df['portPcntPinv'] = 1 - df['portPcnt']
df['portPcntPinv'] = 1 / df['portPcnt']
df['portPcntPinv2'] = df['portPcntPinv'] / df['portPcntPinv'].sum() * 100
df['portAmount'] = df['portPcntPinv2'] * bal / 100
df['portAmount_usd'] = df['portPcntPinv2'] * bal / 100
df['portAmount_units'] = df['portAmount_usd'] / df['price_usd']

c = '24h_volume_usd id market_cap_usd name percent_change_24h percent_change_7d price_btc price_usd portPcnt portPcntPinv portPcntPinv2 portAmount_usd portAmount_units'.split(' ')    
c = '24h_volume_usd name percent_change_24h percent_change_7d price_usd portPcnt portPcntPinv portPcntPinv2 portAmount_usd portAmount_units'.split(' ')    
c = '24h_volume_usd name percent_change_24h percent_change_7d price_usd portPcnt portPcntPinv portPcntPinv2 portAmount_usd portAmount_units'.split(' ')
df = df.sort_values(by='portPcntPinv2', ascending=False)

# tradableCoins2
#print list(tradableCoins.index)
tradableCoins2 = df.set_index('name').ix[list(tradableCoins.index), c]
tradableCoins2 = tradableCoins2.combine_first(tradableCoins)
tradableCoins2 = tradableCoins2[tradableCoins2['portAmount_units'] > 0].sort_values(by='portAmount_units', ascending=False)

with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print '            bal: %s' % bal
    print '   price_usdSUM: %s' % df['price_usd'].sum()
    print '    portPcntSUM: %s' % df['portPcnt'].sum()
    print 'portPcntPinvSUM: %s' % df['portPcntPinv'].sum()
    print 'portAmount_usdSUM: %s' % tradableCoins2['portAmount_usd'].sum()
    print 'portAmount_usd_YoBit_SUM: %s' % tradableCoins2[tradableCoins2['YoBit'] == 1]['portAmount_usd'].sum()

    #tradableCoins2['Poloniex'] = tradableCoins2['Poloniex'] - tradableCoins2['YoBit']
    #print tradableCoins2[tradableCoins2['YoBit'] == 1]
    print tradableCoins2
    #print tradableCoins

    #print df
    print df.ix[:, c]

#df = p.DataFrame(response.data)#.transpose()
#pf(df)
#df    print list(tradableCoins.index)


In [ ]:
from qore import XPath
import pandas as p
xp = XPath()
def getExchanges(coin):
    xresd = xp.xpath2df('http://coinmarketcap.com/currencies/%s/' % coin, {
        'source'       : '//tbody/tr/td[2]/a/text()',
        'pair'         : '//tbody/tr/td[3]/a/text()',
        'volume_24h'   : '//tbody/tr/td[4]/span/text()',
        'price'        : '//tbody/tr/td[5]/span/text()',
        'volume_pcnt'  : '//tbody/tr/td[6]/text()',
        'updated'      : '//tbody/tr/td[7]/text()',
    })
    df = p.DataFrame(xresd)
    df = p.DataFrame(df['source'].drop_duplicates())
    df[coin] = 1
    df = df.set_index('source')
    #print df.transpose()
    df = df.transpose()
    return df

#dfxs = p.DataFrame()
#for i in df['id']:
#    print i
#    dfxs = dfxs.combine_first(getExchanges(i))
#print dfxs.fillna(0)
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    #print df    
    # list most frequent exchanges
    mostFrequentExchanges = p.DataFrame(dfxs.fillna(0).sum()).sort_values(by=0, ascending=False)
    print mostFrequentExchanges
    pdfxs = dfxs.ix[:, list(mostFrequentExchanges.index[0:5])].fillna(0)
    pdfxs = dfxs[dfxs.fillna(0).transpose().sum() > 0].fillna(0)
    tradableCoins = pdfxs.ix[:, list(mostFrequentExchanges.index[0:5])][pdfxs > 0].fillna(0)
    print tradableCoins
    # list coins that trade on most exchanges
    print p.DataFrame(dfxs.fillna(0).transpose().sum()).sort_values(by=0, ascending=False)
#df.index
#df = df.combine_first(getExchanges('1337'))

In [ ]:
df = df.sort_values(by='portPcntPinv2', ascending=False)
#for i in df.index[0:10]: df = df.combine_first(getExchanges(i))
c = '24h_volume_usd name percent_change_24h percent_change_7d price_usd portPcnt portPcntPinv portPcntPinv2 portAmount_usd portAmount_units Poloniex YoBit'.split(' ')
c = 'name price_usd portPcntPinv2 portAmount_usd portAmount_units Poloniex YoBit'.split(' ')
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print df.fillna('').ix[:, c]
#print xresd
#print p.DataFrame(xresd)